Install Required Libraries

In [5]:
!pip install geopandas
!pip install geemap
!pip install earthengine-api

Authenticates Earth Engine

In [6]:
import ee
import geemap

# Authenticate with Earth Engine
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize(project='minaseyedi-internship')

Adjust File Input

In [7]:
from google.colab import drive

drive.mount('/content/drive')

shape_file_path = '/content/drive/GEE_Exe/ArcGIS/Alamdeh.shp'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Modify the SentinelImageDownloader Class

In [14]:
import ee
import geemap
import geopandas as gpd

class SentinelImageDownloader:
  def __init__(self, shapefile_path, start_date, end_date, output_folder):
    self.shapefile_path = shapefile_path
    self.start_date = start_date
    self.end_date = end_date
    self.output_folder = output_folder
    self.aoi = self.load_shapefile()

  def load_shapefile(self):
    # Load the shapefile using GeoPandas
    gdf = gpd.read_file(self.shapefile_path)

    # Reproject the GeoDataFrame to WGS84 (EPSG:4326) for compatibility with Earth Engine
    gdf = gdf.to_crs(epsg=4326)

    # Combine all geometries using union_all instead of unary_union
    geometry = gdf.geometry.union_all()

    # Get the coordinates of the extior boundary
    if geometry.geom_type == 'GeomtryColllection':
      coords = [list(geom.extrior.coords) if geom.geom_type == 'Polygon' else [] for geom in geometry]
      coords = [coord for sublist in coords for coord in sublist]
    else:
      coords = list(geometry.exterior.coords)

    return ee.Geometry.Polygon(coords)

  def get_sentinel1_images(self):
    sentinel1_ImageCollections = ee.ImageCollection("COPERNICUS/S1_GRD") \
      .filterBounds(self.aoi) \
      .filterDate(self.start_date, self.end_date) \
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
      .select(['VV', 'VH'])   # Select 'VV' and 'VH' bands
    count = sentinel1_ImageCollections.size().getInfo()
    print(f"Number of Sentinel-1 images: {count}")

    return sentinel1_ImageCollections


  def get_sentinel2_images(self):
    sentinel2_ImageCollections = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
      .filterBounds(self.aoi) \
      .filterDate(self.start_date, self.end_date) \
      .filter(ee.Filter.lt('cloud_coverage', 30)) \
      .select(['B4', 'B3', 'B2'])

    count = sentinel2_ImageCollections.size().getInfo()
    print(f"Number of Sentinel-2 images: {count}")
    return sentinel2_ImageCollections


  def get_ESA_WorldCover_v100_image(self):
    count = ee.ImageCollection('ESA/WorldCover/v100').size().getInfo()
    print(f"Number of Sentinel-100 images: {count}")
    return ee.ImageCollection('ESA/WorldCover/v100').first()


  def get_ESA_WorldCover_v200_image(self):
    count = ee.ImageCollection('ESA/WorldCover/v200').size().getInfo()
    print(f"Number of Sentinel-200 images: {count}")
    return ee.ImageCollection('ESA/WorldCover/v200').first()


  def download_images(self, image_collection, prefix):
    def download_image_to_drive(image):
        # Export the image as a GeoTIFF to Drive
        image_id = image.get('system:index').getInfo()
        filename = f"{prefix}_{image_id}.tif"
        print(filename)
        task = ee.batch.Export.image.toDrive(
          image=image.clip(self.aoi),
          description=f"export_{prefix}_{image_id}",
          fileNamePrefix=filename,
          region=self.aoi,
          scale=10,
          crs='EPSG:4326'
        )
        task.start()

    # Check if the input is an ImageCollection or a single Image
    if isinstance(image_collection, ee.ImageCollection):
      image_list = image_collection.toList(image_collection.size())  # Get the list of images
      for i in range(image_list.size().getInfo()):  # Iterate through the list
        image = ee.Image(image_list.get(i))
        download_image_to_drive(image)  # Download the image
    elif isinstance(image_collection, ee.Image):
      # If it's a single image, download it directly
      download_image_to_drive(image_collection)

  def run(self):
    sentinel1_image_collection = self.get_sentinel1_images()
    sentinel2_image_collection = self.get_sentinel2_images()
    esa_100_image_collection = self.get_ESA_WorldCover_v100_image()
    esa_200_image_collection = self.get_ESA_WorldCover_v200_image()

    print("Downloading Sentinel-1 images...")
    if sentinel1_image_collection.size().getInfo() > 0:
      self.download_images(sentinel1_image_collection, "S1")
    else:
      print("No Sentinel-1 images found.")

    print("Downloading Sentinel-2 images...")
    if sentinel2_image_collection.size().getInfo() > 0:
      self.download_images(sentinel2_image_collection, "S2")
    else:
      print("No Sentinel-2 images found.")

    print("Downloading ESA v100 images...")
    if esa_100_image_collection:
      self.download_images(esa_100_image_collection, "ESA100")
    else:
      print("No ESA v100 images found.")

    print("Downloading ESA v200 images...")
    if esa_200_image_collection:
      self.download_images(esa_200_image_collection, "ESA200")
    else:
      print("No ESA v200 images found.")


if __name__ == "__main__":
  # Define your parameters
  shapefile_path = '/content/drive/MyDrive/GEE_Exe/ArcGIS/Alamdeh.shp'  # Update the path accordingly
  start_date = "2022-01-01"
  end_date = "2022-03-01"
  output_folder = '/content/drive/MyDrive/GEE_Exe/GEE_Downloads/'  # Save to your Google Drive folder

  # Create an instance of the downloader class
  downloader = SentinelImageDownloader(shapefile_path, start_date, end_date, output_folder)

  # Run the image download process
  downloader.run()


Number of Sentinel-1 images: 13
Number of Sentinel-2 images: 0
Number of Sentinel-100 images: 1
Number of Sentinel-200 images: 1
S1_S1A_IW_GRDH_1SDV_20220105T142758_20220105T142823_041327_04E9C3_B53E.tif
S1_S1A_IW_GRDH_1SDV_20220110T143608_20220110T143633_041400_04EC2C_822E.tif
S1_S1A_IW_GRDH_1SDV_20220111T023716_20220111T023741_041407_04EC61_2594.tif
S1_S1A_IW_GRDH_1SDV_20220117T142758_20220117T142823_041502_04EF78_8FF6.tif
S1_S1A_IW_GRDH_1SDV_20220122T143608_20220122T143633_041575_04F1EA_A52F.tif
S1_S1A_IW_GRDH_1SDV_20220123T023715_20220123T023740_041582_04F228_8A45.tif
S1_S1A_IW_GRDH_1SDV_20220129T142757_20220129T142822_041677_04F566_0804.tif
S1_S1A_IW_GRDH_1SDV_20220203T143607_20220203T143632_041750_04F7DA_0832.tif
S1_S1A_IW_GRDH_1SDV_20220204T023715_20220204T023740_041757_04F817_F3C1.tif
S1_S1A_IW_GRDH_1SDV_20220215T143607_20220215T143632_041925_04FE02_21F4.tif
S1_S1A_IW_GRDH_1SDV_20220216T023715_20220216T023740_041932_04FE41_4609.tif
S1_S1A_IW_GRDH_1SDV_20220227T143607_20220227T1